# Scraping Offical German Government Website
- germany has different federal states and each federal state has kreise and each kreis has an amtsgericht in which the Zwangsversteigerungen take place
- each amtsgericht uploads the zwangsversteigerungen to https://www.zvg-portal.de/index.php?button=Suchen&all=1, this offers only a generale overview
- there are different of information in our data, the general description, the exposee, gutachten and amtliche bekanntmachung and fotos
- we only want to focus on the general description, amtliche bekanntmachung and exposee when scraping, we will safe the gutachten as external link and the fotos, most of the time the fotos arent actually really helpful, because its only pictures from outside the apartment or house and the grundrisse are also meh, but we will safe this as external links, if we want to use this data later
- the gutachten has also some interesting informations, but from the gutachten we get an information overload, what we could try is to feat 
- value they delivier and test especially for Berlin if we get better predictions in our model with these informations (looking into regression by picture or translating a picture into text) 
- then there are are websites which seem to scrape the government website and display it with less information but in a more stylized way like (https://versteigerungspool.de/, https://www.zwangsversteigerung.de/ or https://www.zvg-online.net/), we dont want to scrape such sites, because they offer us no additional value, except for easier scraping in some parts

- https://www.zvg-portal.de/ (official website of the german government where foreclosure auctions can be found for selected regional courts) 
  - (Effort: uniform general description, more detailed information in unstructured PDFs, which would need to be extracted)
  - https://www.zvg-portal.de/index.php?button=Suchen&all=1
  - alle 16 Bundesländer haben einen kürzel auf dieser Website, diese Kürzel kann ich entweder aus https://www.zvg-portal.de/index.php?button=Termine%20suchen entnehmen oder als constante speichern, bevorzugter Weise von der Webseite entnehmen
  - Alerting, wenn für ein Bundesland keine neuen Daten kamen
  - pandas df in sqllite db speichern anstatt in csv
  - PDF Daten müssen vorher schon entweder als string mit in die DB gegeben werden oder am besten schon gleich processen, damit ich nicht zu viele Daten speichere
  - Gutachten, Amtliche Bekanntmachung, Exposee erstmal als external Links speichern
  - How to scrape:
    - go to https://www.zvg-portal.de/index.php?button=Termine%20suchen form -> iterate through table first tr td should contain Land
    - if it contains Land then do: go to next tr.td.select inside select are options take each option value attribute except for 0
    - after having all values do a post request for each saved value https://www.zvg-portal.de/index.php?button=Suchen&all=1 with body ger_name=--+Alle+Amtsgerichte+--&order_by=2&land_abk=value&ger_id=0&az1=&az2=&az3=&az4=&art=&obj=&str=&hnr=&plz=&ort=&ortsteil=&vtermin=&btermin=
      - inside form the second table go through all trs if tr contains three tds, go to the second td, get the a element link 
      - open the link
      - get table first tr contains in first td aktenzeichen and in second td letzte Aktualisierung
      - after first tr go through all other trs if first td in a tr equals to key[: -1] in keywords (keywords={"Art der Versteigerung", "Grundbuch", "Objekt/Lage", "Beschreibung", "Verkehrswert in €", "Termin", "Ort der Versteigerung", "amtliche Bekanntmachung", "Exposee", "Gutachten"}) then safe the value if value contains a then safe the link
      - if key == "Foto" then safe link in list
      - after finishing iteration safe foto list in fotos and append everything saved as new row to an sqlite db 

In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3

In [2]:
# Keywords dictionary for extracting specific table rows
keywords = {
    "Art der Versteigerung", "Grundbuch", "Objekt/Lage", "Beschreibung", 
    "Verkehrswert in €", "Termin", "Ort der Versteigerung", 
    "amtliche Bekanntmachung", "Exposee", "Gutachten"
}

# Create an SQLite connection and a table
conn = sqlite3.connect('foreclosures.db')
cur = conn.cursor()
cur.execute('''
    CREATE TABLE IF NOT EXISTS foreclosure_data (
        aktenzeichen TEXT,
        letzte_aktualisierung TEXT,
        art_der_versteigerung TEXT,
        grundbuch TEXT,
        objekt_lage TEXT,
        beschreibung TEXT,
        verkehrswert TEXT,
        termin TEXT,
        ort_der_versteigerung TEXT,
        amtliche_bekanntmachung TEXT,
        exposee TEXT,
        gutachten TEXT,
        fotos TEXT
    )
''')
conn.commit()

#### Federal state codes
- TODO: Description

In [ ]:
url = "https://www.zvg-portal.de/index.php?button=Termine%20suchen"
response = requests.get(url)
search_soup = BeautifulSoup(response.text, 'html.parser')

land_select = None
for tr in search_soup.find_all('tr'):
    if "Land" in tr.text:
        land_select = tr.find_next('select')
        break

if not land_select:
    print("Land select field not found.")
    exit()

land_codes = [option['value'] for option in land_select.find_all('option') if option['value'] != "0"]

#### Auctions
- TODO: Description

In [ ]:

for land_code in land_codes:
    post_url = "https://www.zvg-portal.de/index.php?button=Suchen&all=1"
    post_data = {
        'ger_name': '--+Alle+Amtsgerichte+--',
        'order_by': '2',
        'land_abk': land_code,
        'ger_id': '0'
    }
    
    post_response = requests.post(post_url, data=post_data)
    if not str(post_response.status_code).startswith("2"):
        print(f"Request for {land_code} failed!")
        continue
    
    land_soup = BeautifulSoup(post_response.text, 'html.parser')
    # TODO folgendes in eine Funktion auslagern
    
    # Iterate over the second table and find the link for each foreclosure case
    result_table = land_soup.find_all('table')[1]  # Second table
    for tr in result_table.find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) == 3:
            link = tds[1].find('a')['href']
            case_url = f"https://www.zvg-portal.de/{link}"
            
            # Open the case link and extract details
            case_response = requests.get(case_url)
            case_soup = BeautifulSoup(case_response.text, 'html.parser')
            
            # Find the first table and extract Aktenzeichen and Letzte Aktualisierung
            details_table = case_soup.find_all('table')[0]
            rows = details_table.find_all('tr')
            aktenzeichen = rows[0].find_all('td')[1].text.strip()
            letzte_aktualisierung = rows[1].find_all('td')[1].text.strip()
            
            # Prepare a dictionary to store the extracted data
            case_data = {
                'aktenzeichen': aktenzeichen,
                'letzte_aktualisierung': letzte_aktualisierung,
                'art_der_versteigerung': None,
                'grundbuch': None,
                'objekt_lage': None,
                'beschreibung': None,
                'verkehrswert': None,
                'termin': None,
                'ort_der_versteigerung': None,
                'amtliche_bekanntmachung': None,
                'exposee': None,
                'gutachten': None,
                'fotos': []
            }
            
            # Iterate through the remaining rows to extract keyword-related data
            for row in rows[1:]:
                first_td = row.find_all('td')[0].text.strip()[:-1]
                if first_td in keywords:
                    value = row.find_all('td')[1].text.strip()
                    case_data[first_td] = value
                    if 'href' in str(row):
                        link = row.find_all('a')[0]['href']
                        case_data[first_td] = link
                    if first_td == "Foto":
                        case_data['fotos'].append(link)
            
            # Save the data in the SQLite database
            cur.execute('''
                INSERT INTO foreclosure_data 
                (aktenzeichen, letzte_aktualisierung, art_der_versteigerung, grundbuch, objekt_lage, beschreibung, 
                verkehrswert, termin, ort_der_versteigerung, amtliche_bekanntmachung, exposee, gutachten, fotos)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                case_data['aktenzeichen'], case_data['letzte_aktualisierung'], case_data['art_der_versteigerung'], 
                case_data['grundbuch'], case_data['objekt_lage'], case_data['beschreibung'], case_data['verkehrswert'], 
                case_data['termin'], case_data['ort_der_versteigerung'], case_data['amtliche_bekanntmachung'], 
                case_data['exposee'], case_data['gutachten'], ','.join(case_data['fotos'])
            ))
            conn.commit()

# Close the database connection
conn.close()
